In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import numpy as np

In [2]:
data = pd.read_csv(r"..\Data\NEW_ADS3.csv")

In [3]:
data = data.drop(['Unnamed: 0', 'finalspend_usd', "effectiveness_usd", "starttime"], axis = 1)
# data = data[data['efficiency']>=0]

In [4]:
data=data[['dag_categ_INDUSTRIAL CAPEX',
       'dag_categ_INDUSTRIAL CAPEX - GLOBAL - ABOVE 500K',
       'dag_categ_INDUSTRIAL CAPEX - LOCAL',
       'dag_categ_Indirect Trade Marketing - Distributors and Associations',
       'dag_categ_Office Supplies', 'dag_categ_Other Office Utilities',
       'dag_categ_Water, Electricity & Gas (Office)', 'categ_l1_COMMERCIAL',
       'categ_l1_PACKAGING', 'categ_l2_IND CAPEX', 'categ_l2_MARKETING',
       'categ_l2_OFFICE UTILITIES', 'categ_l2_PACKAGING', 'categ_l2_POCM',
       'categ_l2_SALES','baselinespend_usd','bestbid_usd','participant','itemno','biddingperiod','timebetweenlotclosing','setareviewperiodafterlotcloses_Yes','zone_APAC','zone_EUR','zone_MAZ','zone_NAZ','zone_SAZ','enabletrafficlightbidding_Yes', "data", "event_id", "efficiency"]]

In [6]:
train = data[data["data"]== 'train']
forecast = data[data['data'] == 'submission_df']

In [7]:
train_final, test_final = train.iloc[:10000], train.iloc[10000:]

In [8]:
train_final

,dag_categ_INDUSTRIAL CAPEX,dag_categ_INDUSTRIAL CAPEX - GLOBAL - ABOVE 500K,dag_categ_INDUSTRIAL CAPEX - LOCAL,dag_categ_Indirect Trade Marketing - Distributors and Associations,dag_categ_Office Supplies,dag_categ_Other Office Utilities,"dag_categ_Water, Electricity & Gas (Office)",categ_l1_COMMERCIAL,categ_l1_PACKAGING,categ_l2_IND CAPEX,...,setareviewperiodafterlotcloses_Yes,zone_APAC,zone_EUR,zone_MAZ,zone_NAZ,zone_SAZ,enabletrafficlightbidding_Yes,data,event_id,efficiency
0,0,0,0,0,0,0,0,1,0,0,...,1,1,0,0,0,0,1,train,FULA,0.312998
1,0,0,1,0,0,0,0,0,0,1,...,1,0,0,1,0,0,1,train,GAEQ,0.019504
2,0,0,1,0,0,0,0,0,0,1,...,1,0,0,0,0,1,1,train,AJUL,0.000000
3,0,0,1,0,0,0,0,0,0,1,...,1,0,0,1,0,0,1,train,CJFZ,0.000000
4,0,0,1,0,0,0,0,0,0,1,...,1,0,0,1,0,0,1,train,BMRW,0.039700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10083,0,0,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,train,CZNG,0.348811
10084,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,train,QAS,0.348000
10085,0,0,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,train,BGME,0.344651
10086,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,train,CURQ,0.344538


In [9]:
train.columns

Index(['dag_categ_INDUSTRIAL CAPEX',
       'dag_categ_INDUSTRIAL CAPEX - GLOBAL - ABOVE 500K',
       'dag_categ_INDUSTRIAL CAPEX - LOCAL',
       'dag_categ_Indirect Trade Marketing - Distributors and Associations',
       'dag_categ_Office Supplies', 'dag_categ_Other Office Utilities',
       'dag_categ_Water, Electricity & Gas (Office)', 'categ_l1_COMMERCIAL',
       'categ_l1_PACKAGING', 'categ_l2_IND CAPEX', 'categ_l2_MARKETING',
       'categ_l2_OFFICE UTILITIES', 'categ_l2_PACKAGING', 'categ_l2_POCM',
       'categ_l2_SALES', 'baselinespend_usd', 'bestbid_usd', 'participant',
       'itemno', 'biddingperiod', 'timebetweenlotclosing',
       'setareviewperiodafterlotcloses_Yes', 'zone_APAC', 'zone_EUR',
       'zone_MAZ', 'zone_NAZ', 'zone_SAZ', 'enabletrafficlightbidding_Yes',
       'data', 'event_id', 'efficiency'],
      dtype='object')

# Modelling with Best Hyperparameters

In [10]:
import optuna
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

def smape(y_true, y_pred):
    epsilon = 1e-7  # Avoid division by zero
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true) + epsilon))

def custom_metric(y_true, y_pred):
    mae_score = mean_absolute_error(y_true, y_pred)
    smape_score = smape(y_true, y_pred)
    return 0.8 * mae_score + 0.2 * smape_score

def catboost_model(target, train, train_full, test, forecast):
    model_name = "CatBoost"
    print(f"Running {model_name}")
    
    x_train = train.drop(target, axis=1)
    y_train = train[target]
    x_train_full = train_full.drop(target, axis=1)
    y_train_full = train_full[target]
    x_test = test.drop(target, axis=1)
    y_test = test[target]
    x_forecast = forecast.drop(target, axis=1)

    # Train and predict with best parameters chnaging depth to 16
    best_params = {
        'iterations': 1000,
        'depth': 16,
        'learning_rate': 0.2885044437918236,
        'l2_leaf_reg': 9.049888644428659,
        'bagging_temperature': 0.3466650286110847,
        'border_count': 173,
        }
    model = CatBoostRegressor(**best_params)  

    # Train on the full dataset and forecast
    model.fit(x_train_full, y_train_full)
    y_forecast = model.predict(x_forecast)
    y_pred = model.predict(x_test)

    print(f"Completed {model_name} model")
    return y_pred, y_forecast


In [12]:
y_pred,y_forecast = catboost_model("efficiency", train_final.drop(['data', 'event_id'],axis = 1), train.drop(['data', 'event_id'],axis = 1),test_final.drop(['data', 'event_id'],axis = 1), forecast.drop(['data', 'event_id'], axis = 1))

Running CatBoost
0:	learn: 0.1159648	total: 1.03s	remaining: 17m 10s
1:	learn: 0.1137083	total: 1.9s	remaining: 15m 48s
2:	learn: 0.1122434	total: 2.89s	remaining: 16m 1s
3:	learn: 0.1104600	total: 3.65s	remaining: 15m 9s
4:	learn: 0.1093384	total: 4.42s	remaining: 14m 39s
5:	learn: 0.1085105	total: 5.2s	remaining: 14m 21s
6:	learn: 0.1076836	total: 6.04s	remaining: 14m 16s
7:	learn: 0.1072436	total: 6.68s	remaining: 13m 48s
8:	learn: 0.1065194	total: 7.33s	remaining: 13m 27s
9:	learn: 0.1063946	total: 7.34s	remaining: 12m 6s
10:	learn: 0.1056913	total: 7.99s	remaining: 11m 57s
11:	learn: 0.1050628	total: 8.63s	remaining: 11m 50s
12:	learn: 0.1045197	total: 9.43s	remaining: 11m 55s
13:	learn: 0.1038505	total: 10.1s	remaining: 11m 50s
14:	learn: 0.1032496	total: 10.7s	remaining: 11m 45s
15:	learn: 0.1029517	total: 11.4s	remaining: 11m 41s
16:	learn: 0.1025617	total: 12.1s	remaining: 11m 37s
17:	learn: 0.1022057	total: 12.7s	remaining: 11m 35s
18:	learn: 0.1015475	total: 13.4s	remaining:

In [13]:
final = pd.concat([forecast['event_id'].reset_index(drop = True), pd.DataFrame(y_forecast, columns = ["efficiency_percentage"])], axis = 1)

In [14]:
# Clipping negative efficiencies
final['efficiency_percentage'] = final['efficiency_percentage'].clip(0.01)

In [15]:
final.to_csv("catboost_optuna2.csv", index=False)